Import Dependencies

In [1]:
import pandas as pd
import os
import datetime as dt
import spmfunctions.spmfunctions as spm
import warnings
warnings.filterwarnings('ignore')

In [2]:
intx='SH-44-Hartley'
intx='us95&30'
suffix=''
bin_len=60#'cycle'
ped=True
hourly=True

f_pkl='df_raw.pk'

yymm=''

if not os.path.exists('./{}/Counts'.format(intx)):
    os.mkdir('./{}/Counts'.format(intx))
    
path_pkl = '.\\{}\\df_raw'.format(intx)

df_raw = pd.read_pickle(os.path.join(path_pkl,f_pkl))
#df_raw=df_raw[df_raw.Coord_plan==13]
df_ct=spm.det_counts(df_raw,'./{}/Configuration/'.format(intx),'dets.csv',bin_len=bin_len,ped=ped)
df_ct.to_csv('./{}/Counts/{}_counts{}{}.csv'.format(intx,intx,yymm,suffix))

In [4]:
dfexc=spm.read_det_cfg('./US95&30/configuration','dets.csv')[1]

In [7]:
dfexc.loc['Status','Det 1'].split(',')

['R']

In [7]:
df_comb = spm.comb_gyr_det(df_raw)

In [11]:
df_phst=spm.phase_status(df_comb)

In [4]:
intersections=[#'Chinden','Colchester', # Error processing Colchester as 15 minute bin, hourly volumes
               'Fairview','Franklin',
               'HobbleCreek','I84EB','I84WB','IslandWoods','McMillan',
               'Pine','RiverValley',
               'StLuke','Ustick',
               'Riverside','Wainwright']#,'Village']

suffix='_15_vph'
bin_len=15
ped=False
combine=False
hourly=True

df_res=pd.DataFrame()
res_dict={}

for intx in intersections:
    if not os.path.exists('./ACHD_Eagle-{}/Counts'.format(intx)):
        os.mkdir('./ACHD_Eagle-{}/Counts'.format(intx))

    print('Starting {} {}'.format(intx,dt.datetime.now()))
    # df_comb
    path_pkl = '.\\ACHD_Eagle-{}\\df_raw'.format(intx)
    
    for f_pkl in [f for f in os.listdir(path_pkl) if f.endswith('pk')]:
        yymm=f_pkl[-7:-3]
        
        df_raw = pd.read_pickle(os.path.join(path_pkl,f_pkl))
        df_ct=spm.det_counts(df_raw,'./ACHD_Eagle-{}/Configuration/'.format(intx),'dets.csv',bin_len=bin_len,ped=ped,hourly=hourly)
        df_ct.to_csv('./ACHD_Eagle-{}/Counts/{}_counts{}{}.csv'.format(intx,intx,yymm,suffix))        
        
        if combine:
            df_res=pd.concat([df_res,df_ct])

    if combine:
        res_dict[intx]=df_res.copy().sort_index()
    
    print('Finished {} {}'.format(intx,dt.datetime.now()))

Starting Fairview 2024-02-14 08:18:22.515442
Finished Fairview 2024-02-14 08:18:53.241130
Starting Franklin 2024-02-14 08:18:53.242129
Finished Franklin 2024-02-14 08:19:24.495986
Starting HobbleCreek 2024-02-14 08:19:24.496491
Finished HobbleCreek 2024-02-14 08:19:39.168971
Starting I84EB 2024-02-14 08:19:39.173656
Finished I84EB 2024-02-14 08:19:56.981667
Starting I84WB 2024-02-14 08:19:56.981667
Finished I84WB 2024-02-14 08:20:20.559152
Starting IslandWoods 2024-02-14 08:20:20.560172
Finished IslandWoods 2024-02-14 08:20:39.994077
Starting McMillan 2024-02-14 08:20:39.995077
Finished McMillan 2024-02-14 08:20:56.144390
Starting Pine 2024-02-14 08:20:56.158660
Finished Pine 2024-02-14 08:21:16.453716
Starting RiverValley 2024-02-14 08:21:16.454680
Finished RiverValley 2024-02-14 08:21:35.781912
Starting StLuke 2024-02-14 08:21:35.786058
Finished StLuke 2024-02-14 08:21:59.063125
Starting Ustick 2024-02-14 08:21:59.064123
Finished Ustick 2024-02-14 08:22:27.366492
Starting Riverside 2

In [93]:
df_res=res_dict['Fairview']
df_res['Ped Total'].sort_values(ascending=False)

Time
2023-11-15 12:00:00    20.0
2023-05-04 20:00:00    18.0
2023-09-23 18:00:00    18.0
2023-10-12 13:00:00    18.0
2023-11-11 17:00:00    18.0
                       ... 
2023-12-31 21:00:00     NaN
2023-12-31 22:00:00     NaN
2023-12-31 22:00:00     NaN
2023-12-31 23:00:00     NaN
2023-12-31 23:00:00     NaN
Name: Ped Total, Length: 19708, dtype: float64

In [100]:
res_dict.keys()

dict_keys(['Chinden', 'Colchester', 'Fairview', 'Franklin', 'HobbleCreek', 'I84EB', 'I84WB', 'IslandWoods', 'McMillan', 'Pine', 'RiverValley', 'StLuke', 'Ustick', 'Riverside', 'Wainwright'])

**Debugging**

In [101]:
df_res=res_dict['Ustick']

with pd.ExcelWriter('./EagleRd Ped Counts.xlsx',
                    mode='w', engine='openpyxl') as writer:

    for intx in res_dict.keys():
    
        df=res_dict[intx].copy()[[c for c in df_res.columns if 'Ped' in c]]
        df=df.groupby([df.index.month, df.index.day_name(),df.index.hour]).mean()
        df.index=df.index.rename(['Month','Day','Hour'])
        df_monthly=df.unstack('Hour')
        df_all=df.groupby(['Day','Hour']).mean().unstack('Hour')
        df_all.index=pd.MultiIndex.from_tuples([('All',day) for day in df_all.index.values], names=["Month", "Day"])
        df_all=pd.concat([df_all,df_monthly])
        
        df_all.to_excel(writer,intx)

In [82]:
# 15-minute
df_res=res_dict['Fairview']

df=df_res.copy()[[c for c in df_res.columns if 'Ped' in c]]
df=df.groupby([df.index.month, df.index.day_name(),df.index.hour,df.index.minute]).mean()
df.index=df.index.rename(['Month','Day','Hour','Minute'])df=df.reset_index()
df.loc[:,'Time']=df.Hour.apply(lambda x: '{:02d}'.format(x))+':'+df.Minute.apply(lambda x: '{:02d}'.format(x))
df=df.loc[:,['Month','Day','Time']+[c for c in df.columns if c.startswith('Ped')]]
df.set_index(['Month','Day','Time']).unstack('Time')

In [52]:
df_monthly=df.unstack('Hour')
df_all=df.groupby(['Day','Hour']).mean().unstack('Hour')
df_all.index=pd.MultiIndex.from_tuples([('All',day) for day in df_all.index.values], names=["Month", "Hour"])
pd.concat([df_all,df_monthly])

ID                  Ped 2                                                    \
Hour                   0         1         2         3         4         5    
Month Hour                                                                    
All   Friday     0.006944  0.004630  0.006944  0.000000  0.004167  0.003704   
      Monday     0.006944  0.006944  0.002315  0.000000  0.004630  0.011574   
      Saturday   0.004299  0.015410  0.009259  0.002315  0.009259  0.011111   
      Sunday     0.012500  0.011111  0.004630  0.004167  0.004167  0.000000   
      Thursday   0.008796  0.004630  0.006481  0.002315  0.012785  0.008796   
...                   ...       ...       ...       ...       ...       ...   
12    Saturday   0.017857  0.017857  0.000000  0.000000  0.000000  0.000000   
      Sunday     0.000000  0.000000  0.000000  0.016667  0.000000  0.000000   
      Thursday   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
      Tuesday    0.000000  0.000000  0.000000  0.000000  0.020833  0.000000   
      Wednesday  0.020833  0.000000  0.000000  0.000000  0.020833  0.000000   

ID                                                       ... Ped Total  \
Hour                   6         7         8         9   ...        14   
Month Hour                                               ...             
All   Friday     0.029907  0.084488  0.097614  0.129274  ...  0.819180   
      Monday     0.006944  0.103426  0.085791  0.125728  ...  0.695833   
      Saturday   0.033141  0.057632  0.101760  0.123316  ...  0.921362   
      Sunday     0.011111  0.055556  0.061064  0.115491  ...  0.832407   
      Thursday   0.019444  0.064744  0.083358  0.155556  ...  0.704630   
...                   ...       ...       ...       ...  ...       ...   
12    Saturday   0.000000  0.000000  0.035714  0.017857  ...  1.071429   
      Sunday     0.000000  0.000000  0.000000  0.050000  ...  0.850000   
      Thursday   0.000000  0.020833  0.083333  0.125000  ...  0.729167   
      Tuesday    0.000000  0.000000  0.020833  0.062500  ...  0.833333   
      Wednesday  0.020833  0.062500  0.145833  0.000000  ...  0.729167   

ID                                                                           \
Hour                   15        16        17        18        19        20   
Month Hour                                                                    
All   Friday     0.924008  0.849339  0.693386  0.600595  0.638889  0.544974   
      Monday     0.811111  0.862037  0.688426  0.566667  0.485185  0.421983   
      Saturday   0.899415  0.866667  0.742848  0.806944  0.557540  0.570833   
      Sunday     0.809259  0.795370  0.551786  0.583995  0.414021  0.373379   
      Thursday   0.759259  0.854002  0.768776  0.596902  0.423504  0.496759   
...                   ...       ...       ...       ...       ...       ...   
12    Saturday   0.894737  0.800000  0.633333  0.433333  0.266667  0.266667   
      Sunday     0.550000  0.500000  0.428571  0.714286  0.267857  0.096154   
      Thursday   0.895833  0.416667  0.562500  0.375000  0.187500  0.354167   
      Tuesday    0.854167  0.958333  0.520833  0.458333  0.270833  0.270833   
      Wednesday  0.854167  0.750000  0.687500  0.291667  0.416667  0.125000   

ID                                             
Hour                   21        22        23  
Month Hour                                     
All   Friday     0.458286  0.309325  0.147215  
      Monday     0.321032  0.191733  0.097685  
      Saturday   0.375860  0.231548  0.121724  
      Sunday     0.258846  0.129843  0.075370  
      Thursday   0.389591  0.255320  0.085176  
...                   ...       ...       ...  
12    Saturday   0.166667  0.166667  0.116667  
      Sunday     0.134615  0.076923  0.020000  
      Thursday   0.062500  0.083333  0.062500  
      Tuesday    0.500000  0.187500  0.208333  
      Wednesday  0.145833  0.229167  0.166667  

[70 rows x 120 columns]